<div id="colab_button">
  <h1>Under the hood</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindai/blob/main/docs/docs/getting-started/under-the-hood.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

### Nitro enclaves with BlindAI API: Under the hood
___________________________________________

In the quick tour, we saw how we can use the BlindAI Whisper API to query the Whisper model with data protected during computation by a Nitro enclave environment.

Let's take a look at what is going on under the hood when we use the Whisper API.

### Deploying the BlindAI Nitro Enclave server
___________________________________________

When you use our Whisper API with Nitro enclaves, by default, you connect to the BlindAI Nitro server hosted by Mithril Security. The advantage of this is that it allows you to test our APIs for free and without spending any time on deployment.

Nitro Enclaves are an AWS technology and can be deployed on Amazon EC2 instances only. Our BlindAI server for Nitro enclaves is deployed on an [Amazon EC2 R6i Instance](https://aws.amazon.com/ec2/instance-types/r6i/).

If you want to deploy your own BlindAI API server for Nitro enclaves, check out our guide on how to do this [here].

 when we query a model hosted using Nitro enclaves using the BlindAI API.

In [ ]:
!pip install requests

import requests

res = requests.post(
    "https://nitro.mithrilsecurity.io/whisper/predict",
    files={
        "audio": open("test2.wav", "rb"),
    },
).text

print(res)

We send our audio file and the query request to the verified Mithril server using a secure TLS communication channel and get back our resulting transcribed text.

In [ ]:
# install AWS CLI
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip awscliv2.zip
!sudo ./aws/install

# check it is now installed
!aws --version

# install terraform
!wget -O- https://apt.releases.hashicorp.com/gpg | sudo gpg --dearmor -o /usr/share/keyrings/hashicorp-archive-keyring.gpg
!echo "deb [signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/hashicorp.list
!sudo apt update && sudo apt install terraform

# check it is now installed 
!terraform --version

Next, you will need to export your AWS credentials in the environment you are working with.

In [ ]:
# !export AWS_SECRET_ACCESS_KEY=[YOUR_AWS_SECRET_ACCESS_KEY_HERE]
# !export AWS_ACCESS_KEY=YOUR_AWS_ACCESS_KEY_HERE]

### Deployment

Now we are ready to deploy our BlindAI server. Before starting, we can check our AWS instance dashboard and see we have no instances currently running

![AWS empty dashboard](../../assets/no-instance-running.png)

We then clone the BlindAI Nitro enclave repo and deploy the server using `terraform apply`.

In [ ]:
# !git clone "https://github.com/mithril-security/whisper-fastapi"
# !cd whisper-fastapi
# !terraform apply

Terraform will produce some output in the terminal confirming the resources that will be allocated to us. 

You will notice in the details of these resources that `enclave_options`: `enabled` is set to `true`.

You must select `yes` to confirm the allocation of these resources.

Terraform then runs our start script to deploy our BlindAI API application on the server instance.

Now when we check out our instance dashboard on our AWS account, we can see our BlindAI Nitro server instance is now running:

![Nitro server running on AWS](../../assets/Nitro-server-running.png)

>You can now get shell access to the application's host machine on port 22. You CANNOT get shell access to the enclave for security reasons.

>You can send requests directly to the enclave, as seen in the previous section, on port 443.

## Conclusions
_________________

In this quick-tour, we have:
- Been introduced to the BlindAI API and seen how we can use the API to query the Whisper model.
- Discovered how queries are performed under the hood.
- Seen how you can deploy your own BlindAI server.

## Coming soon

We hope you'll enjoy testing out our BlindAI API models with Nitro Enclave and Intel SGX TEEs! 

Meanwhile, we are busy preparing for our next step, the release of our **BlindBox** solution, allowing developers to deploy not just **the BlindAI servers**, but ANY compatible application server, within a secure enclave!

## Community
_________________

Ask questions on our [**Github**](https://github.com/mithril-security/blindai/issues) or on our [**Discord**](https://discord.com/invite/TxEHagpWd4). We love feedback!🎉